# Lesson 3. Coordinate Reference Systems (CRS) & Map Projections

## 3.1 California County shapefile

## 3.2 USA State shapefile (or Bike?)

## 3.3 Plot the two together

## 3.4 Coordinate Reference Systems (CRS)

## 3.5 CRS in GeoPandas

## 3.6 Exercise

---
<div style="display:inline-block;vertical-align:middle;">
<a href="https://dlab.berkeley.edu/" target="_blank"><img src ="../assets/images/dlab_logo.png" width="75" align="left">
</a>
</div>

<div style="display:inline-block;vertical-align:middle;">
    <div style="font-size:larger">&nbsp;D-Lab @ University of California - Berkeley</div>
    <div>&nbsp;Team Geo<div>
</div>
        
